In [1]:
import tensorflow as tf

In [2]:
import numpy as np

### unpack 처리하면 실제 데이터를 전치한다

In [3]:
data_ = np.loadtxt('./data.csv', delimiter=',', unpack=False, dtype='float32')

In [4]:
data_

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1.]], dtype=float32)

In [5]:
data_.shape

(6, 5)

In [6]:
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

In [7]:
data

array([[0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

In [8]:
data.shape

(5, 6)

### 두 개의 데이터로 분리한 후에 전치행렬을 저장

In [9]:
x_data = np.transpose(data[0:2])

In [10]:
x_data

array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [11]:
y_data = np.transpose(data[2:])

In [12]:
y_data

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

### 학습에 사용되지 않는 변수 선언

In [13]:
global_step = tf.Variable(0, trainable=False, name='global_step')

Instructions for updating:
Colocations handled automatically by placer.


## 심경망 모델 구성

In [14]:
X = tf.placeholder(tf.float32)

In [15]:
Y = tf.placeholder(tf.float32)

In [16]:
with tf.name_scope('layer1') :
    W1 = tf.Variable(tf.random_uniform([2,10], -1.,1.))
    L1 = tf.nn.relu(tf.matmul(X, W1))

In [17]:
with tf.name_scope('layer12') :
    W2 = tf.Variable(tf.random_uniform([10,20], -1.,1.))
    L2 = tf.nn.relu(tf.matmul(L1, W2))   

In [18]:
with tf.name_scope('output') :
    W3 = tf.Variable(tf.random_uniform([20,3], -1.,1.))
    model = tf.matmul(L2, W3)

In [19]:
with tf.name_scope('optimizer') :
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimzer = tf.train.AdamOptimizer(learning_rate=0.1)
    train_op = optimzer.minimize(cost, global_step=global_step)

In [20]:
tf.summary.scalar('cost', cost)

<tf.Tensor 'cost:0' shape=() dtype=string>

## 신경망 모델 학습 

In [21]:
sess = tf.Session()

In [22]:
saver = tf.train.Saver(tf.global_variables())

In [23]:
ckpt = tf.train.get_checkpoint_state("./model2")

In [24]:
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path) :
    saver.restore(sess, ckpt.model_checkpoint_path)
else :
    sess.run(tf.global_variables_initializer())

In [25]:
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter('./logs/aaa', sess.graph)

In [26]:
for step in range(2) :
    sess.run(train_op, feed_dict = {X : x_data, Y : y_data})
    
    print('Step : %d ' %  sess.run(global_step), 
             'Cost : %.3f ' % sess.run(cost, feed_dict={X : x_data, Y:y_data}))
    
    summary = sess.run(merged, feed_dict={X:x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)

print(' 예측값 : ', sess.run(prediction, feed_dict={ X: x_data}))
print(' 실제값 : ', sess.run(target, feed_dict={Y : y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print(" 정확도  %.2f:" % sess.run(accuracy * 100, feed_dict={X: x_data, Y : y_data}))

Step : 1  Cost : 1.090 
Step : 2  Cost : 0.868 
 예측값 :  [0 2 2 0 0 2]
 실제값 :  [0 1 2 0 0 2]
 정확도  83.33:
